In [1]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

from ssri_interactions.io import load_derived_generic
from ssri_interactions.config import Config
from ssri_interactions.decoding.loaders import  FSFastDecodeDataLoaderTwoWindows
from ssri_interactions.decoding.preprocessors import DecodePreprocessor
from ssri_interactions.decoding.runners import DecodeRunner
from ssri_interactions.decoding.decoders import Decoder
from ssri_interactions.decoding.shuffle import shuffle_X

c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [12]:
import numpy as np
import pandas as pd

def make_fast_ts_runner(session, window_1, window_2, bin_width=0.05):
    loader = FSFastDecodeDataLoaderTwoWindows(
        session_name=session, 
        bin_width=bin_width, 
        window_1=window_1,
        window_2=window_2,
    )
    preprocessor = DecodePreprocessor(
        thresh_empty=1, 
    )
    estimator = LGBMClassifier(n_estimators=40, max_depth=4, num_leaves=30)
    cv = StratifiedKFold(n_splits=5, shuffle=False)
    decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
    runner = DecodeRunner(
        loader=loader,
        preprocessor=preprocessor,
        decoder=decoder,
        nboot=1,
    )
    return runner


In [13]:
neuron_types = load_derived_generic("neuron_types.csv").query("group in ('CIT', 'SAL')")
sessions = neuron_types.query("experiment_name == 'HAMILTON'").session_name.unique().tolist()
session = sessions[0]

#### Initial window versus baseline

In [15]:
window_pre = (-0.5, -0.1)
window_1 = (0.05, 0.2)
window_2 = (0.5, 0.8)


runner_w1 = make_fast_ts_runner(
    session=sessions[1],
    window_1=window_pre,
    window_2=window_1, 
    bin_width=0.1
)
pop_w1, unit_w1 = runner_w1.run_multiple(sessions)

hamilton_01


100%|██████████| 1/1 [00:00<00:00, 10.47it/s]


hamilton_02


100%|██████████| 1/1 [00:00<00:00, 11.26it/s]


hamilton_13


...

100%|██████████| 1/1 [00:00<00:00,  9.26it/s]


hamilton_19


100%|██████████| 1/1 [00:00<00:00,  8.93it/s]


hamilton_20


100%|██████████| 1/1 [00:00<00:00, 10.10it/s]


#### Second window versus baseline

In [16]:
runner_w2 = make_fast_ts_runner(
    session=sessions[1],
    window_1=window_pre,
    window_2=window_2, 
    bin_width=0.1
)
pop_w2, unit_w2 = runner_w2.run_multiple(sessions)

hamilton_01


100%|██████████| 1/1 [00:00<00:00, 10.86it/s]


hamilton_02


100%|██████████| 1/1 [00:00<00:00, 10.74it/s]


hamilton_13


...

100%|██████████| 1/1 [00:00<00:00,  8.13it/s]


hamilton_19


100%|██████████| 1/1 [00:00<00:00,  8.13it/s]


hamilton_20


100%|██████████| 1/1 [00:00<00:00,  8.57it/s]


#### Initial window versus second window

In [17]:
runner_comp = make_fast_ts_runner(
    session=sessions[1],
    window_1=window_1,
    window_2=window_2, 
    bin_width=0.1
)
pop_comp, unit_comp = runner_comp.run_multiple(sessions)

hamilton_01


100%|██████████| 1/1 [00:00<00:00, 12.35it/s]


hamilton_02


100%|██████████| 1/1 [00:00<00:00, 11.91it/s]


hamilton_13


...

100%|██████████| 1/1 [00:00<00:00,  9.43it/s]


hamilton_19


100%|██████████| 1/1 [00:00<00:00,  8.88it/s]


hamilton_20


100%|██████████| 1/1 [00:00<00:00,  9.62it/s]


In [53]:
df_unit_specific = (  
    pd.concat([
    unit_w1.assign(window="w1"), 
    unit_w2.assign(window="w2"), 
    unit_comp.assign(window="comp"),
    ])
    .merge(neuron_types[["session_name", "group"]].drop_duplicates())
)


df_pop_specific = (  
    pd.concat([
    pop_w1.assign(window="w1"), 
    pop_w2.assign(window="w2"), 
    pop_comp.assign(window="comp"),
    ])
    .merge(neuron_types[["session_name", "group"]].drop_duplicates())
)

dd = Config.derived_data_dir / "decoding"
df_unit_specific.to_csv(dd / "fs_fast - unit_specific_windows.csv", index=False)
df_pop_specific.to_csv(dd / "fs_fast - pop_specific_windows.csv", index=False)
